In [ ]:
1) track daily crypto mentions by specific influencers
2) track historical and current crypto trends
3) see if there is a corralation between price spikes and twiitter mentions, and if so how many days it is offset

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

In [113]:
base_url="https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/day/2020-06-01/2020-06-17?apiKey=ui4pRrbHr7mNiqyn6tBCURIIGdQjjWEl"
base_url="https://api.polygon.io/v3/reference/tickers?apiKey=ui4pRrbHr7mNiqyn6tBCURIIGdQjjWEl"

base_url="https://api.polygon.io/v2/aggs/ticker/DASH/prev?adjusted=true&apiKey=ui4pRrbHr7mNiqyn6tBCURIIGdQjjWEl"

base_url="http://rest.coinapi.io/v1/exchangerate/ALICE?apikey=54244F72-7D5F-4173-ADBC-B15B53191244"

base_url="http://rest.coinapi.io/v1/assets?apikey=54244F72-7D5F-4173-ADBC-B15B53191244"


    




In [114]:
b=requests.get(base_url).json()

In [116]:
df = pd.DataFrame(b)
  
# Print the output.
df.loc[df.asset_id=='KMD']

,asset_id,name,type_is_crypto,data_quote_start,data_quote_end,data_orderbook_start,data_orderbook_end,data_trade_start,data_trade_end,data_symbols_count,volume_1hrs_usd,volume_1day_usd,volume_1mth_usd,price_usd,id_icon,data_start,data_end
646,KMD,Komodo,1,2017-03-14T20:10:41.4539303Z,2021-11-27T19:21:45.2721774Z,2017-03-14T20:10:41.4539303Z,2020-08-05T14:37:45.2940000Z,2017-02-06T05:21:43.0000000Z,2021-11-27T19:19:54.4010000Z,62,330227.3,12404506.94,1.472952e+08,1.036924,7349028c-bd7b-499d-be01-1b51bd29ad39,2017-02-06,2021-11-27


In [95]:
df = pd.DataFrame(b['rates'])

In [108]:
df

,time,asset_id_quote,rate
0,2021-11-27T19:31:40.4000000Z,$PAC,2.928526e+03
1,2021-11-27T19:31:40.4000000Z,0X,1.618980e+01
2,2021-11-27T19:31:40.4000000Z,0XBTC,6.669732e+00
3,2021-11-27T19:31:40.4000000Z,100X,4.312145e+09
4,2021-11-27T19:31:40.4000000Z,10SET,1.954161e+01
...,...,...,...
4320,2021-11-27T19:31:40.4000000Z,ZTO,6.363764e-01
4321,2021-11-27T19:31:40.4000000Z,ZUSD,2.029943e+01
4322,2021-11-27T19:31:40.4000000Z,ZWAP,6.644804e-01
4323,2021-11-27T19:31:40.4000000Z,ZYN,8.922169e+01


In [112]:
df.loc[df.asset_id_quote=='KMD']

,time,asset_id_quote,rate
2063,2021-11-27T19:31:40.4000000Z,KMD,18.927185


In [117]:
from bs4 import BeautifulSoup
import tweepy

In [ ]:
twitter crypto influencers

Anthony Pompliano
Erik Voorhees
Ty Smith
Tone Vays
CryptoCobain
Tyler Winklevoss
Vitalik Buterin
CryptoWendyO
StackingUSD
Girl Gone Crypto
Euclid and Oaks
David Gokhshtein
Hailey Lennon
Justin Sun
Ivan on Tech
Kenn Bosak
CryptoBrekkie
Scott Melker
TheCryptoDog
BitBoy Crypto
Dan Held
PeterMcCormack
LayahHeilpern
Coinbound

In [137]:
consumer_key='hNfVZsRGCw053Glc286u62ayY'
consumer_secret='phxQWK2zZLcv6MPINo7cAZv8EdToRCpeu9VryuGi0NOjkx5W77'

In [121]:
# Authorize our Twitter credentials
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

tweets = api.user_timeline(screen_name=userID, 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )

NameError: name 'access_token' is not defined

In [144]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)


In [145]:
try:
    redirect_url = auth.get_authorization_url()
    print('success')
except tweepy.TweepError:
    print('Error! Failed to get request token.')

success


In [146]:
redirect_url

'https://api.twitter.com/oauth/authorize?oauth_token=wPm7-gAAAAABWQJ9AAABfWNEuiE'

In [147]:
api = tweepy.API(auth)
api.update_status('tweepy + oauth!')

Forbidden: 403 Forbidden
453 - You currently have Essential access which includes access to Twitter API v2 endpoints only. If you need access to this endpoint, you’ll need to apply for Elevated access via the Developer Portal. You can learn more here: https://developer.twitter.com/en/docs/twitter-api/getting-started/about-twitter-api#v2-access-leve

In [148]:
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)

In [149]:
auth

In [152]:
tweets = api.user_timeline(screen_name="@LadyofCrypto1", 
                           # 200 is the maximum allowed count
                           count=200,
                           include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )

In [153]:
tweets

[Status(_api=<tweepy.api.API object at 0x7f815547fe80>, _json={'created_at': 'Sat Nov 27 20:36:08 +0000 2021', 'id': 1464694550887374848, 'id_str': '1464694550887374848', 'full_text': "@joecryp98721446 @trader1sz First SZ is a mate. Second, I don't think he called NGL, NAKA, FLAME, NETVRK, UOS, etc. If he did, amazing, because he knows his shit. If he didn't, then suck my dick dumbass", 'truncated': False, 'display_text_range': [28, 202], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'joecryp98721446', 'name': 'joecrypto', 'id': 1347485103484661760, 'id_str': '1347485103484661760', 'indices': [0, 16]}, {'screen_name': 'trader1sz', 'name': 'TraderSZ', 'id': 2837666919, 'id_str': '2837666919', 'indices': [17, 27]}], 'urls': []}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': 1464693087343505413, 'in_reply_to_status_id_str': '1464693087343505413', 'in_reply_to_user_id': 134748510348